In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

# Для работы с матрицами
from scipy.sparse import csr_matrix, coo_matrix

# Детерминированные алгоритмы
from implicit.nearest_neighbours import ItemItemRecommender, CosineRecommender, TFIDFRecommender, BM25Recommender

# Метрики
from implicit.evaluation import train_test_split
from implicit.evaluation import precision_at_k, mean_average_precision_at_k, AUC_at_k, ndcg_at_k

In [6]:
data = pd.read_csv('retail_train.csv')
data.head(2)

,user_id,basket_id,day,item_id,quantity,sales_value,store_id,retail_disc,trans_time,week_no,coupon_disc,coupon_match_disc
0,2375,26984851472,1,1004906,1,1.39,364,-0.6,1631,1,0.0,0.0
1,2375,26984851472,1,1033142,1,0.82,364,0.0,1631,1,0.0,0.0


In [7]:
test_size_weeks = 3

data_train = data[data['week_no'] < data['week_no'].max() - test_size_weeks]
data_test = data[data['week_no'] >= data['week_no'].max() - test_size_weeks]

### Задание 1. Weighted Random Recommendation

Напишите код для случайных рекоммендаций, в которых вероятность рекомендовать товар прямо пропорциональна логарифму продаж
- Можно сэмплировать товары случайно, но пропорционально какому-либо весу
- Например, прямопропорционально популярности. Вес = log(sales_sum товара)

In [11]:
result = data_test.groupby('user_id')['item_id'].unique().reset_index()
result.columns=['user_id', 'actual']
result['actual'] = result['actual'].apply(lambda x: list(x))
result.head()

,user_id,actual
0,1,"[821867, 834484, 856942, 865456, 889248, 90795..."
1,3,"[835476, 851057, 872021, 878302, 879948, 90963..."
2,6,"[920308, 926804, 946489, 1006718, 1017061, 107..."
3,7,"[840386, 889774, 898068, 909714, 929067, 95347..."
4,8,"[835098, 872137, 910439, 924610, 992977, 10412..."


In [8]:
def weighted_random_recommendation(items_weights, n=5):
    """Случайные рекоммендации
    
    Input
    -----
    items_weights: pd.DataFrame
        Датафрейм со столбцами item_id, weight. Сумма weight по всем товарам = 1
    """
    items = items_weights['item_id'].values
    weights = items_weights['weight'].values

    recs = np.random.choice(items, size=n, replace=False, p=weights)

    # Подсказка: необходимо модифицировать функцию random_recommendation()
    # your_code
    
    return recs.tolist()

In [9]:
item_weights = data_train.groupby('item_id')['sales_value'].sum().reset_index()
item_weights['weight'] = item_weights['sales_value']/np.sum(item_weights['sales_value'])
item_weights.head()

,item_id,sales_value,weight
0,25671,20.94,2.969296e-06
1,26081,0.99,1.403822e-07
2,26093,1.59,2.254623e-07
3,26190,1.54,2.183723e-07
4,26355,1.98,2.807644e-07


In [12]:
%%time

result['weighted_random_recommendation'] = result['user_id'].apply(lambda x: weighted_random_recommendation(items_weights=item_weights, n=5))

Wall time: 1.46 s


In [13]:
result.head()

,user_id,actual,weighted_random_recommendation
0,1,"[821867, 834484, 856942, 865456, 889248, 90795...","[1126899, 938512, 997719, 866211, 8090485]"
1,3,"[835476, 851057, 872021, 878302, 879948, 90963...","[870271, 858597, 837569, 871402, 828901]"
2,6,"[920308, 926804, 946489, 1006718, 1017061, 107...","[923746, 1095218, 6704035, 933278, 999563]"
3,7,"[840386, 889774, 898068, 909714, 929067, 95347...","[901470, 1098844, 1022097, 1126899, 871440]"
4,8,"[835098, 872137, 910439, 924610, 992977, 10412...","[1084053, 1037262, 908531, 483681, 945385]"


### Задание 2. Расчет метрик
Рассчитайте Precision@5 для каждого алгоритма с помощью функции из вебинара 1. Какой алгоритм показывает лучшее качество?

In [31]:
import os, sys
    
from metrics import hit_rate_at_k, precision_at_k, recall_at_k, ap_k

In [32]:
result = pd.read_csv('predictions_basic.csv')
result.head(2)

,user_id,actual,random_recommendation,popular_recommendation,itemitem,cosine,tfidf
0,1,[ 821867 834484 856942 865456 889248 ...,"[9222001, 91225, 9251136, 899844, 13095424]","[6534178, 6533889, 1029743, 6534166, 1082185]","[6666, 1082185, 981760, 1127831, 995242]","[1082185, 6666, 981760, 1127831, 1098066]","[1082185, 981760, 1127831, 6666, 1098066]"
1,3,[ 835476 851057 872021 878302 879948 ...,"[950899, 8203686, 875108, 12263268, 997988]","[6534178, 6533889, 1029743, 6534166, 1082185]","[6666, 1082185, 981760, 1098066, 995242]","[1082185, 1098066, 981760, 6666, 826249]","[1082185, 981760, 1098066, 826249, 6666]"


In [41]:
result

,user_id,actual,random_recommendation,popular_recommendation,itemitem,cosine,tfidf
0,1,[ 821867 834484 856942 865456 889248 ...,"[9222001, 91225, 9251136, 899844, 13095424]","[6534178, 6533889, 1029743, 6534166, 1082185]","[6666, 1082185, 981760, 1127831, 995242]","[1082185, 6666, 981760, 1127831, 1098066]","[1082185, 981760, 1127831, 6666, 1098066]"
1,3,[ 835476 851057 872021 878302 879948 ...,"[950899, 8203686, 875108, 12263268, 997988]","[6534178, 6533889, 1029743, 6534166, 1082185]","[6666, 1082185, 981760, 1098066, 995242]","[1082185, 1098066, 981760, 6666, 826249]","[1082185, 981760, 1098066, 826249, 6666]"
2,6,[ 920308 926804 946489 1006718 1017061 ...,"[529703, 915747, 1117401, 2002287, 9526279]","[6534178, 6533889, 1029743, 6534166, 1082185]","[6666, 1082185, 981760, 1127831, 995242]","[1082185, 6666, 981760, 1127831, 1098066]","[1082185, 981760, 1127831, 6666, 878996]"
3,7,[ 840386 889774 898068 909714 929067 ...,"[840890, 1059067, 888236, 970283, 12263185]","[6534178, 6533889, 1029743, 6534166, 1082185]","[6666, 1082185, 981760, 1127831, 995242]","[1082185, 981760, 6666, 1127831, 961554]","[1082185, 981760, 1127831, 6666, 961554]"
4,8,[ 835098 872137 910439 924610 992977 ...,"[956689, 969457, 1109357, 5568675, 72792]","[6534178, 6533889, 1029743, 6534166, 1082185]","[6666, 1082185, 981760, 1127831, 1098066]","[1082185, 981760, 6666, 1098066, 826249]","[1082185, 981760, 6666, 1098066, 826249]"
...,...,...,...,...,...,...,...
2037,2496,[6534178],"[10356686, 6569496, 13775029, 1005496, 13190636]","[6534178, 6533889, 1029743, 6534166, 1082185]","[6666, 1082185, 981760, 1098066, 995242]","[1082185, 981760, 6666, 1098066, 826249]","[1082185, 981760, 1098066, 6666, 826249]"
2038,2497,[ 1016709 9835695 1132298 16809501 845294 ...,"[925544, 1001618, 2991358, 13038819, 10284931]","[6534178, 6533889, 1029743, 6534166, 1082185]","[6666, 1082185, 981760, 1098066, 1127831]","[1082185, 981760, 6666, 1098066, 826249]","[1082185, 981760, 1098066, 6666, 826249]"
2039,2498,[15716530 834484 901776 914190 958382 ...,"[1056926, 96261, 45831, 1128161, 13429421]","[6534178, 6533889, 1029743, 6534166, 1082185]","[6666, 1082185, 981760, 1098066, 1127831]","[1082185, 6666, 981760, 1098066, 826249]","[1082185, 981760, 6666, 1098066, 826249]"
2040,2499,[ 867188 877580 902396 914190 951590 ...,"[1039131, 1153659, 7168968, 7410303, 889198]","[6534178, 6533889, 1029743, 6534166, 1082185]","[6666, 1082185, 981760, 1098066, 995242]","[1082185, 981760, 6666, 1098066, 826249]","[1082185, 981760, 1098066, 6666, 826249]"


In [47]:
for col in result.columns[1:]:
    print(precision_at_k(result[col], result['actual'], k=5))

1.0
0.0
0.0
0.0
0.0
0.0


### Задание 3*. Улучшение бейзлайнов и ItemItem

- Попробуйте улучшить бейзлайны, считая их на топ-5000 товаров
- Попробуйте улучшить разные варианты ItemItemRecommender, выбирая число соседей $K$.

In [ ]:
# your_code